In [ ]:
!pip install anvil-uplink

In [ ]:
import anvil.server
import anvil.media
import torch
from PIL import Image
from torchvision import transforms
from torchvision.io import read_image
model = torch.load('/content/drive/MyDrive/models/model_vit_b_16.pth', map_location=torch.device('cpu'))
anvil.server.connect('JFDXNYXAXBJV23A4KXTKPDXI-URV3BQY2DAOWMB54')

In [ ]:
def predict(img):
  mean = [0.485, 0.456, 0.406] 
  std = [0.229, 0.224, 0.225]
  transform_norm = transforms.Compose([transforms.ToTensor(), 
  transforms.Resize((224,224)),transforms.Normalize(mean, std)])

  img_normalized = transform_norm(img).float()
  img_normalized = img_normalized.unsqueeze_(0)
  dic = {0:'Benign', 1:'Ductal Carcinoma', 2:'Lobular Carcinoma', 3:'Mucinous Carcinoma', 4:'Papillary Carcinoma'}

  with torch.no_grad():
    model.eval()  
    output =model(img_normalized)
    
    index = output.data.numpy().argmax()
    class_name = dic[index]

  return class_name

In [ ]:
@anvil.server.callable
def classify_image(file):
   with anvil.media.TempFile(file) as filename:
      img = read_image(filename)
      
   return predict(img)